# Preprocesamiento de los Datos - Pipeline

A continuación se crea un pipeline que tiene como propósito ser utilizado para limpiar los datos de forma general antes de pasarlos a cualquier limpieza necesaria para cualquier algoritmo en particular. Se realizan, entre otros, ejercicios de one-hot encoding, imputación y normalizaión. Todo esto se consolida en un Pipeline para su reutilización.

## 0. Instalar dependencias

In [ ]:
%%sh
pip install -r requirements.txt

## 1. Carga y exploración de los datos

In [3]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
#Cargar el archivo csv a un DataFrame
df = pd.read_csv('./data/Datos_BiciAlpes.csv')

NameError: name 'pd' is not defined

'/Users/elefantickets/laboratory-01'